# Use Mistral in Azure AI and Azure ML

Use `mistralai` client to consume Mistral deployments in Azure AI and Azure ML. Notice that Mistral supports only chat completions API.

> Review the [documentation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-mistral) for the Mistral family of models at for AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Follow the steps listed in [this](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-mistral?tabs=mistral-large#prerequisites) article to set up resources.
* Go to Azure AI Studio and select the model on Model Catalog.

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Create a Serverless deployment using the steps listed [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-mistral?tabs=mistral-large#create-a-new-deployment).

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-mistral?tabs=mistral-large) for model deployment and inference.

To complete this tutorial, you will need to:

* Install `mistralai`:

    ```bash
    pip install mistralai
    ```

## Example

The following is an example about how to use `mistralai` with a Mistral model deployed in Azure AI and Azure ML:

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

To use `mistralai`, create a client and configure it as follows:

- `endpoint`: Use the endpoint URL from your deployment. Do not include either `/chat/completions` as this is included automatically by the client.
- `api_key`: Use your API key.

In [ ]:
client = MistralClient(
    endpoint="https://<endpoint>.<region>.inference.ai.azure.com", api_key="<secret>"
)

> Tip: Alternatively, you can configure your API key in the environment variables `MISTRAL_API_KEY`.

Use the client to create chat completions requests:

In [ ]:
chat_response = client.chat(
    model="azureai",
    messages=[
        ChatMessage(
            role="user",
            content="Who is the most renowned French painter? Provide a short answer.",
        )
    ],
    max_tokens=50,
)

The generated text can be accessed as follows:

In [ ]:
print(chat_response.choices[0].message.content)

Mistral also support the parameter `safe_prompt`. Toggling the safe prompt will prepend your messages with the following system prompt:

> Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

In [ ]:
chat_response = client.chat(
    model="azureai",
    messages=[
        ChatMessage(
            role="user",
            content="Who is the most renowned French painter? Provide a short answer.",
        )
    ],
    max_tokens=50,
    safe_prompt=True,
)

The generated text can be accessed as follows:

In [ ]:
print(chat_response.choices[0].message.content)

### Tools

Certain versions of the Mistral model family support function calling using Tools. `Mistral Large` for instance has such capability.

The following example shows you can use function calling with `Mistral Large` to get flight information between cities. Let's define the tools available for the model:

In [ ]:
from mistralai.models.chat_completion import Function, ToolChoice

tools = [
    {
        "type": "function",
        "function": Function(
            name="get_flight_info",
            description="Get flight information between two cities or airports",
            parameters={
                "type": "object",
                "properties": {
                    "loc_origin": {
                        "type": "string",
                        "description": "The departure airport, e.g. MIA",
                    },
                    "loc_destionation": {
                        "type": "string",
                        "description": "The destination airport, e.g. NYC",
                    },
                },
                "required": ["loc_origin", "loc_destination"],
            },
        ),
    }
]

Once the toolkit is designed, we can just past it to the LLM call:

In [ ]:
messages = [
    ChatMessage(
        role="system",
        content="You are a helpful assistant that help users to find information about traveling, how to get to places and the different transportations options. You care about the environment and you always have that in mind when answering inqueries.",
    ),
    ChatMessage(
        role="user",
        content="When is the next flight from Miami to Seattle?",
    ),
]

chat_response = client.chat(
    model="azureai", messages=messages, tools=tools, tool_choice=ToolChoice.auto
)

You can find out if a tool needs to be called by inspecting the response. When a tool needs to be called, you will see `finish_reason` is `FinishReason.tool_calls`

In [ ]:
print(chat_response.choices[0].finish_reason)

response_message = chat_response.choices[0].message
tool_calls = response_message.tool_calls
print(tool_calls)

To get the details of the tool that needs to be called, you can inspect the repsonse. The following example gets and prints all the tools being needed.

In [ ]:
import json

for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    tool_call_id = tool_call.id

    print(f"Calling function `{function_name}` with arguments {function_args}")

To demostrate usage, let's define this function to be implemented in Python directly as a `function`. We are going to mock a simple response but you can call an external service, or provide any output.

In [ ]:
def get_flight_info(loc_origin: str, loc_destination: str):
    return "No flights available. You should take a train, specially if it helps to reduce CO2 emissions."

Since this function is defined in the current notebook, we can get the function by anme using `locals()` dictionary. Notice that this is just done as a simple way to get the function callable from its string name. Then we can call it with the corresponding arguments.

In [ ]:
callable_func = locals()[function_name]
function_response = callable_func(**function_args)
print(function_response)

Once we have a response from the function and its arguments, we can append a new message to the chat history. Notice how we are indicating the model that this chat message came from a tool:

In [ ]:
messages.append(
    {
        "tool_call_id": tool_call_id,
        "role": "tool",
        "name": function_name,
        "content": function_response,
    }
)

Let's see the response from the model now:

In [ ]:
chat_response = client.chat(
    model="azureai",
    messages=messages,
    tools=tools,
)
print(chat_response.choices[0].message.content)

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)